### Set path to original pyNeuroChem. Please change to your own path

#### Import required libraries

In [1]:
import sys
#sys.path.append('/home/olexandr/notebooks/ASE_ANI/lib')
from ase_interface import ANIENS
from ase_interface import aniensloader

van Der Waals correction will be unavailable. Please install ased3


In [2]:
import numpy as np
import  ase
#from ase.build import molecule
from ase.neb import NEB
from ase.calculators.mopac import MOPAC
from ase.optimize import BFGS
from ase.neb import NEBtools
from ase.io import read, write
from ase.neb import NEBTools
import matplotlib
import matplotlib as mpl

import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline

In [3]:
# Read the previous configurations
initial = read('data/ethane_dehydro_initial.xyz')
final = read('data/ethane_dehydro_final.xyz')

#### Load the 8x network ANI-1ccx ensemble
Preloading the ensemble allows a single class to be used in a for loop or multiple classes without being reloaded

In [4]:
# Current ANI model options are:
# '../ani_models/ani-1ccx_8x.info' Coupled cluster transfer learned model
# '../ani_models/ani-1x_8x.info'   Full ANI-1x wb97x/6-31g* dataset model
aniensemble = aniensloader('../ani_models/ani-1ccx_8x.info', 0, multigpu=False)

In [5]:
#  Make 13 images (note the use of copy)
configs = [initial.copy() for i in range(13)] + [final.copy()]

configsM = [initial.copy() for i in range(13)] + [final.copy()]

In [6]:
for config in configs:
    # Current ANI model options are:
    # '../ani_models/ani-1ccx_8x.info' Coupled cluster transfer learned model
    # '../ani_models/ani-1x_8x.info'   Full ANI-1x wb97x/6-31g* dataset model
    config.set_calculator(ANIENS(aniensemble))
    
for configM in configsM:
    configM.set_calculator(MOPAC())

In [7]:
# Make the NEB object, interpolate to guess the intermediate steps
neb = NEB(configs, k=5.0)
neb.interpolate()

# Make the NEB object, interpolate to guess the intermediate steps
nebM = NEB(configsM, k=1.0)
nebM.interpolate()

In [8]:
#initial.get_potential_energy()

In [9]:
relax = BFGS(neb, trajectory='neb.traj')
relax.run(fmax=1.0)

      Step     Time          Energy         fmax
BFGS:    0 16:28:19    -2161.994555        7.1799
BFGS:    1 16:28:19    -2162.510286        4.6784
BFGS:    2 16:28:19    -2162.853694        4.0589
BFGS:    3 16:28:19    -2163.105898        3.5775
BFGS:    4 16:28:19    -2163.331578        3.1969
BFGS:    5 16:28:19    -2163.588207        3.2600
BFGS:    6 16:28:20    -2163.681854        3.8968
BFGS:    7 16:28:20    -2163.706213        4.1037
BFGS:    8 16:28:20    -2163.705384        4.4267
BFGS:    9 16:28:20    -2163.697481        3.9793
BFGS:   10 16:28:20    -2163.692890        2.1429
BFGS:   11 16:28:20    -2163.699929        1.8136
BFGS:   12 16:28:20    -2163.713912        1.6600
BFGS:   13 16:28:20    -2163.728968        1.3541
BFGS:   14 16:28:20    -2163.742827        1.2571
BFGS:   15 16:28:21    -2163.755718        0.8777


True

In [10]:
relaxM = BFGS(nebM, trajectory='nebM.traj')
relaxM.run(fmax=1.0)

RuntimeError: mopac in . returned an error: 127

In [ ]:
CH1_dist = []
CH2_dist = []
E_rel = []
# Compare intermediate steps to initial energy
initial.set_calculator(ANI())
e0 = initial.get_potential_energy()
for config in configs:
    d1 = config[0].position - config[2].position
    d2 = config[1].position - config[5].position
    CH1_dist.append(np.linalg.norm(d1))
    CH2_dist.append(np.linalg.norm(d2))
    E_rel.append(config.get_potential_energy()- e0)
    #print(np.linalg.norm(d1), config.get_potential_energy() - e0)

In [ ]:
CH1_distM = []
CH2_distM = []
E_relM = []
# Compare intermediate steps to initial energy
initial.set_calculator(MOPAC())
e0 = initial.get_potential_energy()
for configM in configsM:
    d1 = configM[0].position - configM[2].position
    d2 = configM[1].position - configM[5].position
    CH1_distM.append(np.linalg.norm(d1))
    CH2_distM.append(np.linalg.norm(d2))
    E_relM.append(configM.get_potential_energy()- e0)

In [ ]:
mpl.rcParams['figure.figsize'] = (10.0, 7.0)
plt.plot(CH1_dist, np.array(E_rel)*23.06, label='ANI')
plt.plot(CH1_distM, np.array(E_relM)*23.06,  label='MOPAC')
plt.legend(fontsize=24)
sns.set(font_scale=1.0)
plt.tick_params(axis='both', which='major', labelsize=20)
plt.xlabel(r'$ R_{o} \ (\AA)$', fontsize=28)
plt.ylabel(r'$ \Delta E\ \mathrm{(kcal/mol)}$', fontsize=28)

plt.title("Ethane dehydrogynation NEB", fontsize=40)
plt.show()